In [1]:
import re
import os
import pickle
import numpy as np
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate
from data.configs.demo import config
from torch.utils.data import DataLoader

In [2]:
from utils.MIND import MIND_bert, MIND

In [3]:
train_path = "C:/MIND/MINDdemo_train/"
config.embedding = 'bert'

config.signal_length = 20
config.his_size = 10
a = MIND_bert(config, train_path + 'news.tsv', train_path + 'behaviors.tsv')

In [4]:
config.embedding = 'glove'

config.his_size = 10
b = MIND(config, train_path + 'news.tsv', train_path + 'behaviors.tsv')

In [5]:
a[10]

{'user_index': array([10]),
 'cdd_id': array([45975, 33900,     0,     0,     0]),
 'his_id': array([21323, 18967, 18909, 15911,  4045, 21717,  8587, 24359, 14485,
        16311]),
 'cdd_encoded_index': array([[10973,  4202,  1005,  1055,  3426,  1997,  2331,  3936,  1010,
          3986,  2326,  5115,  2005,  2101,  2023,  3204,  1996,  9971,
          2351,  2012],
        [ 2798,  7369,  1010,  2280,  4174,  2110,  2374,  1010,  5626,
          7212,  2732,  1010,  2757,  2012,  4229,  2798,  7369,  1010,
          1996,  2280],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,    

In [6]:
b[10]

{'user_index': array([10]),
 'cdd_id': array([45975, 33900,     0,     0,     0]),
 'his_id': array([21323, 18967, 18909, 15911,  4045, 21717,  8587, 24359, 14485,
        16311]),
 'cdd_encoded_index': array([[8840, 1003,   11, 1254,    9,  307, 1511,    4, 1394,  315, 1237,
           12,  598,   31,  322,    1,    1,    1,    1,    1],
        [1803, 2838,    4,  124,  219,   55,  120,    4,  371,  556,  391,
            4,  433,   16, 2920,    1,    1,    1,    1,    1],
        [   1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1],
        [   1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1],
        [   1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1]]),
 'his_encoded_index': array([[ 2637,  5755,    11,   798,  1406,    22,  3509,   582,   777,
        

In [7]:
from torch.utils.data.distributed import DistributedSampler
s = DistributedSampler(a, num_replicas=2, rank=0, shuffle=False)

In [8]:
loader_train = DataLoader(a, batch_size=config.batch_size, pin_memory=False, num_workers=0, drop_last=False, shuffle=False, collate_fn=my_collate, sampler=None)
record = next(iter(loader_train))

In [8]:
loader_train = DataLoader(b, batch_size=config.batch_size, pin_memory=False, num_workers=0, drop_last=False, shuffle=False, collate_fn=my_collate, sampler=None)
record = next(iter(loader_train))

In [11]:
record['cdd_attn_mask'].shape, record['cdd_encoded_index'].shape

(torch.Size([10, 5, 20]), torch.Size([10, 5, 20]))